# Excel Engine - Extended

Download the zip file from the button below and unzip it into your pycode directory.
After you unzip that file, we willl read the analysis results
We can do this with three lines of code.

In [1]:
%load_ext autoreload
%autoreload 2

# My Footing Designs

Let's say we need three footing sizes and our job now is to group the factored reactions into working with one of three footing types.
This is where we get to do _engineering_! 
Let's manually review our reactions and get a feel for the forces we are dealing with.
Then, let's estimate some footing sizes that we will use for our design.

In [2]:
from typing import Optional
import csv

def str_to_float(x: str) -> float:
    try:
        return float(x)
    except:
        return x

def parse_table(filepath: str) -> list[list[str]]:
    """
    Returns a list[list[str]] representing data in a table. All numeric values
    are converted to floats
    """
    with open(filepath) as file:
        line_acc = []
        for line in csv.reader(file):
            cell_acc = []
            for cell in line:
                cell_acc.append(str_to_float(cell))
            line_acc.append(cell_acc)
    return line_acc


def extract_table_columns(
    table_data: list[list[str | float]],
    column_indexes: list[int], 
    column_headers: Optional[list[str]] = None
) -> dict[str, list[str | float]]:
    """
    Returns a dictionary of column values extracted from 'table_data'

    'table_data': A list of lists representing a table by records
    'column_indexes': A list of integers representing the columns to extract from each record
    'column_headers': A list of str representing what the keys of the dictionary should be for each column
        If not supplied, the column headers are used as keys.
    """
    table_data_no_headers = table_data[1:]
    transposed_table = list(zip(*table_data_no_headers))
    extraction_acc = {}
    if column_headers is None:
        column_headers = table_data[0]
    for pos_idx, column_index in enumerate(column_indexes):
        extraction_acc.update({column_headers[pos_idx]: transposed_table[column_index]})
    return extraction_acc
    

# def create_condition_check(check_against_value: float, op: str) -> callable:
#     """
#     Returns a function with a single numerical input parameter.
#     The function returns a boolean corresponding to whether the 
#     single numerical argument passed to it meets the condition
#     encoded in the function.

#     'check_against_value' the value that will be encoded in the function
#         to check against.
#     'op': str, one of {"ge", "le", "gt", "lt", "eq", "ne"}
#     """
#     operators = {
#         "ge": operator.ge,
#         "le": operator.le,
#         "gt": operator.gt,
#         "lt": operator.lt,
#         "eq": operator.eq,
#         "ne": operator.ne,
#     }
#     def checker(test_value):
#         return operators[op](test_value, check_against_value)

#     return checker
    

# def excel_runner(
#     xlsx_filepath,
#     demand_input_cell_arrays: dict[str, list],
#     identifier_cell_arrays: dict[str, list],
#     design_inputs: dict[str, dict[str, float]],
#     result_cells: list[str],
#     save_conditions: dict[str, callable],
#     save_dir: Optional[str] = None,
#     sheet_idx: int = 0
# ) -> None:
#     demand_cell_ids = list(demand_input_cell_arrays.keys())
#     iterations = len(demand_input_cell_arrays[demand_cell_ids[0]])
#     for iteration in range(iterations):
#         demand_cells_to_change = {cell_id: demand_input_cell_arrays[cell_id][iteration] for cell_id in demand_cell_ids}
#         for design_tag, design_cells_to_change in design_inputs.items():
#             cells_to_change = demand_cells_to_change | design_cells_to_change
#             calculated_results = excel_engine(
#                 xlsx_filepath, 
#                 cells_to_change=cells_to_change,
#                 cells_to_retrieve=result_cells,
#                 sheet_idx=sheet_idx
#             )
        
#             save_condition_acc = []
#             for idx, result_cell_id in enumerate(result_cells):
#                 calculated_result = calculated_results[idx]
#                 save_condition_acc.append(save_conditions[result_cell_id](calculated_result))
            
#             if all(save_condition_acc):
#                 filepath = pathlib.Path(xlsx_filepath)
#                 name = filepath.stem
#                 suffix = filepath.suffix
#                 demand_ids = "-".join([id_array[iteration] for id_array in identifier_cell_arrays])
                
#                 new_filename = f"{name}-{demand_ids}-{design_tag}.{suffix}"
#                 save_dir_path = pathlib.Path(save_dir)
#                 if not save_dir_path.exists():
#                     save_dir_path.mkdir(parents=True)
#                 calculated_results = excel_engine(
#                     xlsx_filepath, 
#                     cells_to_change=cells_to_change,
#                     cells_to_retrieve=result_cells,
#                     sheet_idx=sheet_idx,
#                     new_filename=f"{str(save_dir)}/{new_filename}"
#                 )
#                 break

In [3]:
from xl_engine.excel_engine import create_condition_check, excel_runner

In [4]:
table_data = parse_table("Model Reactions Export - SI.csv")
demand_data = extract_table_columns(table_data, [8, 6, 10], ["E44", "E46", "E45"])
identifier_data = extract_table_columns(table_data, [1, 5])
dcr_check = create_condition_check(1.0, "le")
footing_sizes = {
    "F1": {"E24": 2500, "E25": 2500, "E14": 400},
    "F2": {"E24": 4000, "E25": 4000, "E14": 600},
    "F3": {"E24": 6500, "E25": 6500, "E14": 800},
}
cells_to_retrieve = ["E33", "E72"]

save_conditions = {cell_id: dcr_check for cell_id in cells_to_retrieve}

excel_runner(
    r"C:\Users\conno\Desktop\Eccentric Concrete Footing.xlsx",
    demand_data,
    identifier_data,
    design_inputs=footing_sizes,
    result_cells=cells_to_retrieve,
    save_conditions=save_conditions,
    save_dir = r"C:\Users\conno\Desktop\design",
)



Output()

(-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', 'The disk is full.', 'xlmain11.chm', 0, -2146827284), 
None)

RuntimeError: An error occured with the Excel interface during saving. Possible causes include:
- You do not have permissions to save to the chosen location.
- Your hard-drive is full.
